In [1]:
# default_exp protein_intensity_estimation

In [2]:
%reload_ext autoreload

%autoreload 2

In [3]:
#export
import pandas as pd
import numpy as np
import directlfq.normalization as lfqnorm
import multiprocessing
import itertools

def estimate_protein_intensities(normed_df, min_nonan, num_samples_quadratic):
    "derives protein pseudointensities from between-sample normalized data"
    
    allprots = list(normed_df.index.get_level_values(0).unique())
    print(f"{len(allprots)} prots total")
    
    list_of_tuple_w_protein_profiles_and_shifted_peptides = get_list_of_tuple_w_protein_profiles_and_shifted_peptides(allprots, normed_df, num_samples_quadratic, min_nonan)
    protein_df2 = get_protein_dataframe_from_list_of_protein_profiles(allprots=allprots, list_of_tuple_w_protein_profiles_and_shifted_peptides=list_of_tuple_w_protein_profiles_and_shifted_peptides, normed_df= normed_df)
    ion_df2 = get_ion_intensity_dataframe_from_list_of_shifted_peptides(list_of_tuple_w_protein_profiles_and_shifted_peptides)

    return protein_df2, ion_df2


def get_list_of_tuple_w_protein_profiles_and_shifted_peptides(allprots, normed_df, num_samples_quadratic, min_nonan):
    pool = get_configured_multiprocessing_pool()
    input_specification_tuplelist_idx__df__num_samples_quadratic__min_nonan = get_input_specification_tuplelist_idx__df__num_samples_quadratic__min_nonan(normed_df, allprots, num_samples_quadratic, min_nonan)
    list_of_tuple_w_protein_profiles_and_shifted_peptides = pool.starmap(calculate_peptide_and_protein_intensities, input_specification_tuplelist_idx__df__num_samples_quadratic__min_nonan)
    pool.close()

    return list_of_tuple_w_protein_profiles_and_shifted_peptides

def get_configured_multiprocessing_pool():
    multiprocessing.freeze_support()
    num_cores = multiprocessing.cpu_count() if multiprocessing.cpu_count() < 61 else 61 #windows upper thread limit
    pool = multiprocessing.Pool(num_cores)
    print(f"using {pool._processes} processes")
    return pool


def get_input_specification_tuplelist_idx__df__num_samples_quadratic__min_nonan(normed_df, allprots, num_samples_quadratic, min_nonan):
    list_of_normed_dfs = get_normed_dfs(normed_df, allprots)
    return zip(range(len(list_of_normed_dfs)),list_of_normed_dfs, itertools.repeat(num_samples_quadratic), itertools.repeat(min_nonan))


def get_normed_dfs(normed_df, allprots):
    list_of_normed_dfs = []
    for protein in allprots:
        peptide_intensity_df = pd.DataFrame(normed_df.loc[protein])#DataFrame definition to avoid pandas Series objects
        peptide_intensity_df = ProtvalCutter(peptide_intensity_df, maximum_df_length=100).get_dataframe()
        list_of_normed_dfs.append(peptide_intensity_df)

    return list_of_normed_dfs


def get_ion_intensity_dataframe_from_list_of_shifted_peptides(list_of_tuple_w_protein_profiles_and_shifted_peptides):
    ion_ints = [x[1] for x in list_of_tuple_w_protein_profiles_and_shifted_peptides]
    ion_df = 2**pd.concat(ion_ints)
    ion_df = ion_df.replace(np.nan, 0)
    return ion_df
    

def get_protein_dataframe_from_list_of_protein_profiles(allprots, list_of_tuple_w_protein_profiles_and_shifted_peptides, normed_df):
    index_list = []
    profile_list = []

    list_of_protein_profiles = [x[0] for x in list_of_tuple_w_protein_profiles_and_shifted_peptides]
    
    for idx in range(len(allprots)):
        if list_of_protein_profiles[idx] is None:
            continue
        index_list.append(allprots[idx])
        profile_list.append(list_of_protein_profiles[idx])
    
    index_for_protein_df = pd.Index(data=index_list, name="protein")
    protein_df = 2**pd.DataFrame(profile_list, index = index_for_protein_df, columns = normed_df.columns)
    protein_df = protein_df.replace(np.nan, 0)
    return protein_df


def calculate_peptide_and_protein_intensities(idx,peptide_intensity_df , num_samples_quadratic, min_nonan):
    if(idx%100 ==0):
        print(f"prot {idx}")
    summed_pepint = np.nansum(2**peptide_intensity_df)
    
    if(peptide_intensity_df.shape[1]<2):
        shifted_peptides = peptide_intensity_df
    else:
        shifted_peptides = lfqnorm.NormalizationManagerProtein(peptide_intensity_df, num_samples_quadratic = num_samples_quadratic).complete_dataframe
    
    protein_profile = get_protein_profile_from_shifted_peptides(shifted_peptides, summed_pepint, min_nonan)
    
    return protein_profile, shifted_peptides



In [4]:
#export
def get_protein_profile_from_shifted_peptides(normalized_peptide_profile_df, summed_pepints, min_nonan):
    intens_vec = get_list_with_protein_value_for_each_sample(normalized_peptide_profile_df, min_nonan)
    intens_vec = np.array(intens_vec)
    summed_intensity = np.nansum(2**intens_vec)
    if summed_intensity == 0: #this means all elements in intens vec are nans
        return None
    intens_conversion_factor = summed_pepints/summed_intensity
    scaled_vec = intens_vec+np.log2(intens_conversion_factor)
    return scaled_vec

def get_list_with_protein_value_for_each_sample(normalized_peptide_profile_df, min_nonan):
    intens_vec = []
    for sample in normalized_peptide_profile_df.columns:
        reps = normalized_peptide_profile_df.loc[:,sample].to_numpy()
        nonan_elems = sum(~np.isnan(reps))
        if(nonan_elems>=min_nonan):
            intens_vec.append(np.nanmedian(reps))
        else:
            intens_vec.append(np.nan)
    return intens_vec


In [5]:
#export
import pandas as pd
from numba import njit

class ProtvalCutter():
    def __init__(self, protvals_df, maximum_df_length = 100):
        self._protvals_df = protvals_df
        self._maximum_df_length = maximum_df_length
        self._dataframe_too_long = None
        self._sorted_idx = None
        self._check_if_df_too_long_and_sort_index_if_so()


    def _check_if_df_too_long_and_sort_index_if_so(self):
        self._dataframe_too_long =len(self._protvals_df.index)>self._maximum_df_length
        if self._dataframe_too_long:
            self._determine_nansorted_df_index()

    def _determine_nansorted_df_index(self):
        idxs = self._protvals_df.index
        self._sorted_idx =  sorted(idxs, key= lambda idx : self._get_num_nas_in_row(self._protvals_df.loc[idx].to_numpy()))
        
    @staticmethod
    @njit
    def _get_num_nas_in_row(row):
        return sum(np.isnan(row))


    def get_dataframe(self):
        if self._dataframe_too_long:
            return self._get_shortened_dataframe()
        else:
            return self._protvals_df

    def _get_shortened_dataframe(self):
        shortened_index = self._sorted_idx[:self._maximum_df_length]
        return self._protvals_df.loc[shortened_index]



### Unit Tests

#### Classes for testcase generation

In [6]:
import numpy as np
import pandas as pd

from  numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

class ProteinProfileGenerator():
    def __init__(self, peptide_profiles):
        self._peptide_profiles = peptide_profiles
        
        self.protein_profile_dataframe = None
        self._generate_protein_profile_dataframe()

    def _generate_protein_profile_dataframe(self):
        collected_profiles = [x.peptide_profile_vector for x in self._peptide_profiles]
        protnames_for_index = [x.protein_name for x in self._peptide_profiles]
        pepnames_for_index = [f'{idx}' for idx in range(len(self._peptide_profiles))]
        self.protein_profile_dataframe = pd.DataFrame(collected_profiles,index=[protnames_for_index, pepnames_for_index])
        self.protein_profile_dataframe = np.log2(self.protein_profile_dataframe.replace(0, np.nan))



class PeptideProfile():
    def __init__(self, protein_name, fraction_zeros_in_profile, systematic_peptide_shift, add_noise, num_samples = 20, min_intensity = 1e6, max_intensity = 1e10):


        self._fraction_zeros_in_profile = fraction_zeros_in_profile
        self._systematic_peptide_shift = systematic_peptide_shift
        self._add_noise = add_noise
        self._min_intensity = min_intensity
        self._max_intensity = max_intensity
        self._num_samples = num_samples

        self.protein_name = protein_name
        self.peptide_profile_vector = []
        self._define_peptide_profile_vector()

    def _define_peptide_profile_vector(self):
        self.peptide_profile_vector = self._get_single_peptide_profile_template()
        self._scale_profile_vector()
        if self._add_noise:
            self._apply_poisson_noise_to_profilevector()
        self._add_zeros_to_profilevector()

    def _get_single_peptide_profile_template(self):
        rs = RandomState(MT19937(SeedSequence(42312)))
        return rs.randint(low=self._min_intensity, high=self._max_intensity,size=self._num_samples)

    def _scale_profile_vector(self):
        self.peptide_profile_vector = self.peptide_profile_vector*self._systematic_peptide_shift

    def _apply_poisson_noise_to_profilevector(self):
        self.peptide_profile_vector = np.random.poisson(lam=self.peptide_profile_vector, size=len(self.peptide_profile_vector))

    def _add_zeros_to_profilevector(self):
        num_elements_to_set_zero = int(self._num_samples*self._fraction_zeros_in_profile)
        idxs_to_set_zero = np.random.choice(self._num_samples,size=num_elements_to_set_zero, replace=False)
        self.peptide_profile_vector[idxs_to_set_zero] = 0
        


#### Tests

In [7]:
import pandas as pd
import numpy as np
#test df cutting

def test_sorting_by_num_nans():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['P', 'P', 'P'],['A', 'B', 'C']])
    pcutter = ProtvalCutter(df,maximum_df_length=2)
    sorted_idx = pcutter._sorted_idx
    df_sorted = df.loc[sorted_idx]
    display(df)
    display(df_sorted)
    assert np.allclose(df_sorted.iloc[2].to_numpy(), vals1,equal_nan=True)
    assert np.allclose(df_sorted.iloc[0].to_numpy(), vals3,equal_nan=True)
    

def test_cutting_of_df():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C']])
    pcutter = ProtvalCutter(df, maximum_df_length=2)
    cut_df = pcutter.get_dataframe()
    ion_idx = [x[0] for x in cut_df.index]
    print(ion_idx)
    assert ion_idx == ['C', 'B']





test_sorting_by_num_nans()
test_cutting_of_df()

0    1    2   3
P A  9.0  NaN  NaN NaN
  B  5.0  6.0  NaN NaN
  C  1.0  2.0  3.0 NaN

0    1    2   3
P C  1.0  2.0  3.0 NaN
  B  5.0  6.0  NaN NaN
  A  9.0  NaN  NaN NaN

['C', 'B']


In [8]:
def test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=False)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=3, add_noise=False)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=0.1, add_noise=False)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=100, add_noise=False)
    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    display(protein_df)
    normed_ion_profile = lfqnorm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,11].dropna().to_numpy()
    display(column_from_shifted)
    assert np.allclose(column_from_shifted, column_from_shifted[0])
    
test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other()



0          1          2          3          4          5   \
protA 0  44.602965        NaN  41.489085  43.216307  43.505708        NaN   
      1        NaN        NaN        NaN  33.250522        NaN        NaN   
      2  29.730290  29.601566  26.616410  28.343632  28.633033  27.465729   
      3        NaN        NaN        NaN        NaN        NaN        NaN   

                6          7          8          9          10         11  \
protA 0  43.454211  43.102621  41.897534  43.625508  43.261340  40.203349   
      1        NaN  33.136837        NaN        NaN        NaN        NaN   
      2  28.581537  28.229946  27.024859  28.752833  28.388665  25.330674   
      3        NaN        NaN        NaN        NaN        NaN        NaN   

                12         13         14         15         16         17  \
protA 0  44.447364  44.554027  44.692343  44.701690  43.106703  44.630975   
      1        NaN        NaN        NaN        NaN        NaN        NaN   
      2        NaN  29.681352  29.819668  29.829015  28.234029  29.758301   
      3        NaN        NaN        NaN  39.794799        NaN  39.724085   

                18         19  
protA 0  43.380650  43.407413  
      1        NaN        NaN  
      2  28.507975        NaN  
      3        NaN        NaN

0          1          2          3          4          5   \
protA 0  44.602965        NaN  41.489085  43.216307  43.505708        NaN   
      1        NaN        NaN        NaN  43.216307        NaN        NaN   
      2  44.602965  44.474241  41.489085  43.216307  43.505708  42.338404   
      3        NaN        NaN        NaN        NaN        NaN        NaN   

                6          7          8          9         10         11  \
protA 0  43.454211  43.102621  41.897534  43.625508  43.26134  40.203349   
      1        NaN  43.102621        NaN        NaN       NaN        NaN   
      2  43.454211  43.102621  41.897534  43.625508  43.26134  40.203349   
      3        NaN        NaN        NaN        NaN       NaN        NaN   

                12         13         14        15         16         17  \
protA 0  44.447364  44.554027  44.692343  44.70169  43.106703  44.630975   
      1        NaN        NaN        NaN       NaN        NaN        NaN   
      2        NaN  44.554027  44.692343  44.70169  43.106703  44.630975   
      3        NaN        NaN        NaN  44.70169        NaN  44.630975   

               18         19  
protA 0  43.38065  43.407413  
      1       NaN        NaN  
      2  43.38065        NaN  
      3       NaN        NaN

array([40.20334853, 40.20334853])

In [9]:
def test_that_profiles_with_noise_are_close():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    
    normed_ion_profile = lfqnorm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,9].dropna().to_numpy()

    assert np.allclose(column_from_shifted, column_from_shifted[0],rtol=0.01, atol=0.01)


test_that_profiles_with_noise_are_close()

0          1          2          3          4          5   \
protA 0  44.602965  44.474241  41.489085  43.216307  43.505707  42.338404   
      1  44.602957  44.474247  41.489092  43.216306  43.505713  42.338412   
      2  44.602982  44.474207  41.489115  43.216292  43.505718  42.338245   
      3  44.602963  44.474241  41.489076  43.216307  43.505707  42.338397   

                6          7          8          9          10         11  \
protA 0  43.454211  43.102621  41.897535  43.625508  43.261339  40.203350   
      1  43.454201  43.102629  41.897508  43.625505  43.261341  40.203282   
      2  43.454228  43.102554  41.897342  43.625404  43.261393  40.203304   
      3  43.454208  43.102621  41.897538  43.625509  43.261343  40.203343   

                12         13         14         15         16         17  \
protA 0  44.447364  44.554028  44.692343  44.701690  43.106703  44.630975   
      1  44.447371  44.554030  44.692343  44.701695  43.106691  44.630963   
      2  44.447351  44.554034  44.692357  44.701691  43.106803  44.630971   
      3  44.447369  44.554027  44.692346  44.701691  43.106703  44.630974   

                18         19  
protA 0  43.380650  43.407414  
      1  43.380654  43.407390  
      2  43.380640  43.407486  
      3  43.380651  43.407411

In [10]:
import directlfq.protein_intensity_estimation as intensity_estimation

def test_that_protein_intensities_are_retained():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    
    peptide_profiles = [peptide1, peptide2, peptide3, peptide4]
    summed_intensity_protein = sum([np.nansum(x.peptide_profile_vector) for x in peptide_profiles])
    
    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    
    protein_df_normed, _ = intensity_estimation.estimate_protein_intensities(protein_df, min_nonan=1, num_samples_quadratic=100)
    display(protein_df_normed)
    summed_lfq_intensities = np.sum(protein_df_normed.iloc[0].to_numpy())
    assert np.allclose(summed_lfq_intensities, summed_intensity_protein)


test_that_protein_intensities_are_retained()

1 prots total
using 10 processes
prot 0
prot 0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
protein,,,,,,,,,,,,,,,,,,,,
protA,2.625387e+13,2.401286e+13,3.032623e+12,1.004078e+13,1.227118e+13,5.463766e+12,1.184089e+13,9.279948e+12,4.025015e+12,1.333366e+13,1.035920e+13,1.243873e+12,2.356961e+13,2.537825e+13,2.793164e+13,2.811340e+13,9.306236e+12,2.676862e+13,1.125228e+13,1.146296e+13


In [11]:
import directlfq.protein_intensity_estimation as intensity_estimation

def run_with_multiple_proteins():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protB",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide5= PeptideProfile(protein_name="protC",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide6= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide7= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide8= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    peptide_profiles = [peptide1, peptide2, peptide3, peptide4, peptide5, peptide6, peptide7, peptide8]
    protein_df = ProteinProfileGenerator(peptide_profiles).protein_profile_dataframe
    protein_df_normed, _ = intensity_estimation.estimate_protein_intensities(protein_df, min_nonan=1, num_samples_quadratic=100)
    display(protein_df_normed)
    
run_with_multiple_proteins()

4 prots total
using 10 processes
prot 0
prot 0
prot 3prot 1

prot 2


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
protein,,,,,,,,,,,,,,,,,,,,
protA,2.334962e+13,2.135609e+13,2.697152e+12,8.930062e+12,1.091349e+13,4.859376e+12,1.053102e+13,8.253311e+12,3.579775e+12,1.185869e+13,9.213088e+12,1.106268e+12,2.096214e+13,2.257069e+13,2.484174e+13,2.500323e+13,8.276699e+12,2.380722e+13,1.000728e+13,1.019482e+13
protB,8.906537e+11,8.146272e+11,1.028823e+11,3.406307e+11,4.162959e+11,1.853566e+11,4.016976e+11,3.148192e+11,1.365508e+11,4.523403e+11,3.514301e+11,4.219842e+10,7.995923e+11,8.609482e+11,9.475776e+11,9.537373e+11,3.157108e+11,9.081151e+11,3.817302e+11,3.888772e+11
protC,8.906551e+11,8.146272e+11,1.028822e+11,3.406304e+11,4.162959e+11,1.853575e+11,4.016974e+11,3.148193e+11,1.365500e+11,4.523405e+11,3.514303e+11,4.219799e+10,7.995903e+11,8.609482e+11,9.475763e+11,9.537370e+11,3.157105e+11,9.081132e+11,3.817307e+11,3.888768e+11
protD,2.671963e+12,2.443880e+12,3.086454e+11,1.021891e+12,1.248889e+12,5.560715e+11,1.205097e+12,9.444565e+11,4.096518e+11,1.357021e+12,1.054292e+12,1.265938e+11,2.398776e+12,2.582846e+12,2.842734e+12,2.861208e+12,9.471345e+11,2.724347e+12,1.145188e+12,1.166632e+12


## Learning tests

In [22]:
import pandas as pd
import numpy as np

def test_that_dataframe_is_generated_as_expected():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=['A', 'A', 'A'])
    display(df)
    assert df.iloc[2, 2] == 11
    assert df.iloc[1, 2] == 7



test_that_dataframe_is_generated_as_expected()

,0,1,2,3
A,1,2,3,4
A,5,6,7,8
A,9,10,11,12


In [23]:
def test_retrieval_of_numpy_arrays_from_dataframe():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C'], ['a', 'b', 'a']])
    display(df)
    assert np.allclose(vals2, df.loc['B'])
    assert np.allclose([2, 6, 10], df.loc[:,1])

test_retrieval_of_numpy_arrays_from_dataframe()

,,0,1,2,3
A,a,1,2,3,4
B,b,5,6,7,8
C,a,9,10,11,12


In [12]:
def test_setting_numpy_seed():
    from numpy.random import MT19937
    from numpy.random import RandomState, SeedSequence

    rs = RandomState(MT19937(SeedSequence(42)))
    res = rs.randint(10,size=20)
    display(res)

test_setting_numpy_seed()

array([2, 6, 8, 8, 3, 3, 3, 3, 4, 7, 2, 7, 5, 4, 0, 8, 1, 3, 7, 1])